# Convolutional Neural Networks

## Project: Write an Algorithm for Landmark Classification

---

In this notebook, some template code has already been provided for you, and you will need to implement additional functionality to successfully complete this project. You will not need to modify the included code beyond what is requested. Sections that begin with **'(IMPLEMENTATION)'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section, and the specifics of the implementation are marked in the code block with a 'TODO' statement. Please be sure to read the instructions carefully! 

> **Note**: Once you have completed all the code implementations, you need to finalize your work by exporting the Jupyter Notebook as an HTML document. Before exporting the notebook to HTML, all the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to **File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut.  Markdown cells can be edited by double-clicking the cell to enter edit mode.

The rubric contains _optional_ "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. If you decide to pursue the "Stand Out Suggestions", you should include the code in this Jupyter notebook.

---
### Why We're Here

Photo sharing and photo storage services like to have location data for each photo that is uploaded. With the location data, these services can build advanced features, such as automatic suggestion of relevant tags or automatic photo organization, which help provide a compelling user experience. Although a photo's location can often be obtained by looking at the photo's metadata, many photos uploaded to these services will not have location metadata available. This can happen when, for example, the camera capturing the picture does not have GPS or if a photo's metadata is scrubbed due to privacy concerns.

If no location metadata for an image is available, one way to infer the location is to detect and classify a discernible landmark in the image. Given the large number of landmarks across the world and the immense volume of images that are uploaded to photo sharing services, using human judgement to classify these landmarks would not be feasible.

In this notebook, you will take the first steps towards addressing this problem by building models to automatically predict the location of the image based on any landmarks depicted in the image. At the end of this project, your code will accept any user-supplied image as input and suggest the top k most relevant landmarks from 50 possible landmarks from across the world. The image below displays a potential sample output of your finished project.

![Sample landmark classification output](images/sample_landmark_output.png)


# The Road Ahead

We break the notebook into separate steps.  Feel free to use the links below to navigate the notebook.

* [Step 0](#step0): Download Datasets and Install Python Modules
* [Step 1](#step1): Create a CNN to Classify Landmarks (from Scratch)
* [Step 2](#step2): Create a CNN to Classify Landmarks (using Transfer Learning)
* [Step 3](#step3): Write Your Landmark Prediction Algorithm

<a id='step0'></a>
## Step 0: Download Datasets and Install Python Modules

**Note: if you are using the Udacity workspace, *YOU CAN SKIP THIS STEP*. The dataset can be found in the `/data` folder and all required Python modules have been installed in the workspace.**

Download the [landmark dataset](https://udacity-dlnfd.s3-us-west-1.amazonaws.com/datasets/landmark_images.zip).
Unzip the folder and place it in this project's home directory, at the location `/landmark_images`. The landmark images are a subset of the **Google Landmarks Dataset v2**.

Install the following Python modules:
* cv2
* matplotlib
* numpy
* PIL
* torch
* torchvision

In [ ]:
#import cv2
#import PIL
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets, models
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler

import os
import random as rd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
torch.cuda.list_gpu_processes(), torch.cuda.memory_snapshot()

In [ ]:
import zipfile
from google.colab import drive
drive.mount('/content/google')

zipfile.ZipFile('/content/google/MyDrive/Colab Notebooks/Udacity/deep_learning/landmark_images.zip').extractall()

In [ ]:
#@title Upload .csvs for train and test data 
upload = True

if upload:
    try:
        train_shapes = pd.read_csv('train_shapes.csv',header=0, index_col=0)
        test_shapes = pd.read_csv('test_shapes.csv',header=0, index_col=0)
    except FileNotFoundError:
        print ("Must upload files first.")

else:
    #read train image file data:
    import cv2
    train_path = './landmark_images/train/'
    cat_path = [train_path + cat + '/' for cat in os.listdir(train_path)]
    jpg_list = [[cp + jpg for jpg in os.listdir(cp)] for cp in cat_path]

    train_images = []
    for jl in jpg_list:
        for jpg in jl:
            train_images.append(jpg)

    train_image_shapes = []
    for imfile in train_images:
        train_image_shapes.append(cv2.imread(imfile).shape)
    heights = [shp[0] for shp in train_image_shapes]    
    widths = [shp[1] for shp in train_image_shapes]
    
    train_images_df = pd.DataFrame(columns=['filepath','height','width'])
    train_images_df['filepath'] = train_images
    train_images_df['height'] = heights
    train_images_df['width'] = widths
    train_images_df.to_csv('train_shapes.csv')
    
    #read test image file data:
    test_path = './landmark_images/test/'
    cat_path = [test_path + cat + '/' for cat in os.listdir(test_path)]
    jpg_list = [[cp + jpg for jpg in os.listdir(cp)] for cp in cat_path]

    test_images = []
    for jl in jpg_list:
        for jpg in jl:
            test_images.append(jpg)

    test_image_shapes = []
    for imfile in test_images:
        test_image_shapes.append(cv2.imread(imfile).shape)
    heights = [shp[0] for shp in test_image_shapes]    
    widths = [shp[1] for shp in test_image_shapes] 

    test_images_df = pd.DataFrame(columns=['filepath','height','width'])
    test_images_df['filepath'] = test_images
    test_images_df['height'] = heights
    test_images_df['width'] = widths
    test_images_df.to_csv('test_shapes.csv')

    #cleanup
    del train_images, train_image_shapes, train_path
    del cat_path, jpg_list, widths, heights
    del test_images, test_image_shapes, test_path
    #del train_images_df, test_images_df

In [ ]:
critA = train_shapes["width"] >= 700
critB = train_shapes["height"] >= 444
#critC = train_shapes["height"] <= 700.loc[critC]
train_shapes["height"].hist()
train_shapes.loc[critA].loc[critB]["height"].hist()

critD = test_shapes["width"] >= 700
critE = test_shapes["height"] >= 444
#critF = test_shapes["height"] <= 700.loc[critF]
test_shapes["height"].hist()
test_shapes.loc[critD].loc[critE]["height"].hist()

#### Preprocessing

In [ ]:
#@title Channel change
def bgr2rgb(im_bgr):
    # because cv2.imread() returns BGR image format
    im_rgb = im_bgr.copy()
    im_rgb[:,:,0] = im_bgr[:,:,2]
    im_rgb[:,:,1] = im_bgr[:,:,1]
    im_rgb[:,:,2] = im_bgr[:,:,0]
    return im_rgb

In [ ]:
#import cv2

def quick_label(row):
    label = row['filepath'][-40:-20] + "\nWidth: " + str(row['width']) + "  Height: " + str(row['height'])
    return label

choices = rd.sample(list(train_shapes.index), 4)
paths = [(train_shapes.iloc[s])['filepath'] for s in choices]
labels = [quick_label(train_shapes.iloc[s]) for s in choices]
imgs_rgb = [plt.imread(p) for p in paths]
#imgs_rgb = [bgr2rgb(im) for im in imgs_bgr] 

fig, axes = plt.subplots(figsize=(18,16), nrows=2, ncols=2)
axes = axes.reshape(4)
for ii, im in enumerate(imgs_rgb):
    ax0 = axes[ii] 
    ax0.set_title(labels[ii])
    ax0.imshow(im)


### (IMPLEMENTATION) Specify Data Loaders for the Landmark Dataset

Use the code cell below to create three separate [data loaders](http://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader): one for training data, one for validation data, and one for test data. Randomly split the images located at `landmark_images/train` to create the train and validation data loaders, and use the images located at `landmark_images/test` to create the test data loader.

All three of your data loaders should be accessible via a dictionary named `loaders_scratch`. Your train data loader should be at `loaders_scratch['train']`, your validation data loader should be at `loaders_scratch['valid']`, and your test data loader should be at `loaders_scratch['test']`.

You may find [this documentation on custom datasets](https://pytorch.org/docs/stable/torchvision/datasets.html#datasetfolder) to be a useful resource.  If you are interested in augmenting your training and/or validation data, check out the wide variety of [transforms](http://pytorch.org/docs/stable/torchvision/transforms.html?highlight=transform)!

In [ ]:
# image directory location
data_dir = '/content/landmark_images'

# set target height and width for all images
H = 480
W = 600
h1 = 525
w1 = 700
h0 = 600
w0 = 800

# Define transforms for the training data and testing data   
# 


train_transforms = transforms.Compose([ transforms.Resize((h0, w0)),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.RandomResizedCrop((h1,w1)),
                                        transforms.RandomRotation( 10, 
                                                                   expand=True, 
                                                                   fill=200),
                                        transforms.CenterCrop((H, W)),
                                        #transforms.Normalize([0.5,0.5,0.5],[0.25,0.25,0.25]),
                                        transforms.ToTensor()]) 
#transforms.Resize((h0, w0)),
#transforms.CenterCrop((h1, w1)),
test_transforms = transforms.Compose([ transforms.Resize((H, W)),
                                       transforms.ToTensor()])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

######### from "Multi-Layer Perceptron, MNIST" notebook #######
# obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(0.2 * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
test_sampler = RandomSampler(test_data)
###############################################################

trainloader = torch.utils.data.DataLoader(train_data, 
                                          batch_size=50,
                                          sampler=train_sampler)

validloader = torch.utils.data.DataLoader(train_data, 
                                          batch_size=25,
                                          sampler=valid_sampler)

testloader = torch.utils.data.DataLoader(test_data, 
                                         batch_size=25,
                                         sampler=test_sampler)

## Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes

loaders_scratch = {'train': trainloader, 'valid': validloader, 'test': testloader}

In [ ]:
## if only want approx 600x800 images
trainable_df = train_shapes.loc[critA].loc[critB]#.loc[critC]
testable_df = test_shapes.loc[critD].loc[critE]#.loc[critF]

**Question 1:** Describe your chosen procedure for preprocessing the data. 
- How does your code resize the images (by cropping, stretching, etc)?  What size did you pick for the input tensor, and why?
- Did you decide to augment the dataset?  If so, how (through translations, flips, rotations, etc)?  If not, why not?

**Answer**: 

In [ ]:
def custom_weight_init(m):
    ## TODO: implement a weight initialization strategy
    for p in m.parameters():
        #n = p.size(0)
        torch.nn.init.normal_(p, mean=0.75, std=1.0/(n**0.5))
        #torch.nn.init.normal_(p, mean=0.66, std=0.22)

<a id='step1'></a>
## Step 1: Create a CNN to Classify Landmarks (from Scratch)

In this step, you will create a CNN that classifies landmarks.  You must create your CNN _from scratch_ (so, you can't use transfer learning _yet_!), and you must attain a test accuracy of at least 20%.

Although 20% may seem low at first glance, it seems more reasonable after realizing how difficult of a problem this is. Many times, an image that is taken at a landmark captures a fairly mundane image of an animal or plant, like in the following picture.

<img src="images/train/00.Haleakala_National_Park/084c2aa50d0a9249.jpg" alt="Bird in Haleakalā National Park" style="width: 400px;"/>

Just by looking at that image alone, would you have been able to guess that it was taken at the Haleakalā National Park in Hawaii?

An accuracy of 20% is significantly better than random guessing, which would provide an accuracy of just 2%. In Step 2 of this notebook, you will have the opportunity to greatly improve accuracy by using transfer learning to create a CNN.

Remember that practice is far ahead of theory in deep learning.  Experiment with many different architectures, and trust your intuition.  And, of course, have fun!

### (IMPLEMENTATION) Visualize a Batch of Training Data

Use the code cell below to retrieve a batch of images from your train data loader, display at least 5 images simultaneously, and label each displayed image with its class name (e.g., "Golden Gate Bridge").

Visualizing the output of your data loader is a great way to ensure that your data loading and preprocessing are working as expected.

In [ ]:
## Two class dicts

classes = test_data.classes
classes.sort()
class2num = {c[3:]:int(c[:2]) for c in classes}
num2class = {int(c[:2]):c[3:] for c in classes}

#classvals = list(set([(str(pair[0]),pair[1]) for pair in test_shapes[['labnum','label']].values]))
#num2class = {int(x):y for x,y in classvals}
#assert class2num[num2class[7]] == 7

In [ ]:
## TODO: visualize a batch of the train data loader
## the class names can be accessed at the `classes` attribute
## of your dataset object (e.g., `train_dataset.classes`)

try:
    images, labels = train_data_iter.next()
except NameError:
    loader = loaders_scratch['train']
    train_data_iter = iter(loader)
    images, labels = train_data_iter.next()

fig = plt.figure(figsize=(21,8),clear=True)
axes = fig.subplots(nrows=2, ncols=4, sharex=True, sharey=True)
axes = axes.flatten()
for i, im in enumerate(images[0:8]):
    ax = axes[i]
    title = num2class[labels[i].item()]
    ax.set_title(title)
    axes[i].set_xticks([])
    axes[i].set_yticks([])
    ax.imshow(np.transpose(im, (1,2,0)))


In [ ]:
### View test images
try:
    images, labels = test_data_iter.next()
except NameError:
    test_data_iter = iter(testloader)
    images, labels = test_data_iter.next()

labels = labels.tolist()

fig = plt.figure(figsize=(22,8.5))#, dpi=72)
axes = fig.subplots(nrows=2, ncols=4)
axes = axes.flatten()

idx = rd.randrange(images.shape[0]-8)
for i in range(idx,idx+8):
    img = np.transpose(images[i], (1,2,0))  #rechannel(img)
    title = num2class[labels[i]]
    axes[i-idx].set_title(title)
    axes[i-idx].set_xticks([])
    axes[i-idx].set_yticks([])
    axes[i-idx].imshow(img)


In [ ]:
img.min(), img.max(), img.mean(), img.std()

### (IMPLEMENTATION) Specify Loss Function and Optimizer

Use the next code cell to specify a [loss function](http://pytorch.org/docs/stable/nn.html#loss-functions) and [optimizer](http://pytorch.org/docs/stable/optim.html).  Save the chosen loss function as `criterion_scratch`, and fill in the function `get_optimizer_scratch` below.

In [ ]:
## TODO: select loss function
criterion_scratch = nn.CrossEntropyLoss()

def get_optimizer_scratch(model):
    ## TODO: select and return an optimizer
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
    return optimizer

### (IMPLEMENTATION) Model Architecture

Create a CNN to classify images of landmarks.  Use the template in the code cell below.

In [ ]:
# Calculate a layer's output shape
def output_volume(w, f, s, p):
    #params:
    #w = input volume (tuple)
    #f = kernel size (tuple)
    #s = stride (int)
    #p = zero padding (int)
    x = (w[0] - f[0] + 2*p)/s + 1
    y = (w[1] - f[1] + 2*p)/s + 1
    return ( int(np.floor(x)), int(np.floor(y)) )

In [ ]:
#@title Original Net (no running)
# define the CNN architecture
if False:
    class oNet(nn.Module):
        ## TODO: choose an architecture, and complete the class
        def __init__(self):
            super(oNet, self).__init__()
            ## Define layers of a CNN
            # number of nodes in each hidden layer
            nodes_1 = 2048
            nodes_2 = 512
            # input layer (H * W -> nodes_1)
            self.fc1 = nn.Linear(H * W, nodes_1)
            # hidden layer (nodes_1 -> nodes_2)
            self.fc2 = nn.Linear(nodes_1, nodes_2)
            # output layer (nodes_2 -> 10)
            self.fc3 = nn.Linear(nodes_2, 10)
            # dropout layers
            self.dropout2 = nn.Dropout(0.2)
            self.dropout3 = nn.Dropout(0.3)
            self.dropout4 = nn.Dropout(0.4)
            
        def forward(self, x):
            ## Define forward pass
            # x is input image tensor
            # flatten input
            x = x.view(-1, H * W)
            # first hidden layer
            x = F.relu(self.fc1(x))
            # first dropout layer
            x = self.dropout3(x)
            # second hidden layer
            x = F.relu(self.fc2(x))
            # second dropout layer
            x = self.dropout2(x)
            # output layer
            x = self.fc3(x)
            return x

    #-#-# Do NOT modify the code below this line. #-#-#

    # instantiate the CNN
    model_scratch = oNet()

    # move tensors to GPU if CUDA is available
    if use_cuda:
        model_scratch.cuda()

In [ ]:
print("input:", W, H)

(w,h) = output_volume((W,H),(50,50),5,25) #1 
print("shrink1 out:",w, h,"\tshrink2 in:",w, h)

(w,h) = output_volume((w,h),(30,30),1,3) #1 
print("shrink2 out:",w, h,"\t1rst in:",w, h)

(w,h) = output_volume((w,h),(11,11),1,1) #1 
print("1rst out:",w, h,"\t2nd in:",w//2, h//2)

(w,h) = output_volume((w//2,h//2),(7,7),1,1) #2
print("2nd out:",w, h,"\t3rd in:",w//2, h//2)

(w,h) = output_volume((w//2,h//2),(5,5),1,1) #3
print("3rd out:", w, h,"\t4th in:",w//2, h//2)

(w,h) = output_volume((w//2,h//2),(3,3),1,1) 
print("4th out:", w, h,"\tfinal out:",w, h)
print("flattened:", w*h)

#@title altered CNN architecture
class cNet(nn.Module):
    def __init__(self):
        super(cNet, self).__init__()

        self.shrink1 = nn.Conv2d(3, 3, 50, stride=5, padding=25)
        self.shrink2 = nn.Conv2d(3, 3, 30, stride=1, padding=3)

        # (138x98x3 image tensor) --> conv1 
        # --> 138,98 - 11 + 2 + 1 = 130,90        
        self.conv1 = nn.Conv2d(3, 20, 11, padding=1)
        #  --> pool /2 --> (65x45x20) --> conv2
        self.conv2 = nn.Conv2d(20, 30, 7, padding=1)
        # 65,45 - 7 + 2 + 1 = 61,41
        # --> pool /2 -->  (30x20x30) --> conv3
        self.conv3 = nn.Conv2d(30, 40, 5, padding=1)
        # 30,20 - 5 + 2 + 1 = 28,18
        # --> pool /2 --> 17,12 (14x9x40) --> conv4
        self.conv4 = nn.Conv2d(40, 50, 3, padding=1)
        # 14,9 - 3 + 2 + 1 = 14,9
        # (14x9x50)=6300 --> flatten

        # max pooling --> conv_layer/2
        self.pool = nn.MaxPool2d(2)

        # dense layers (14 * 9 * 50 -> 6300)
        self.dense1 = nn.Linear(6300, 2100)
        self.dense2 = nn.Linear(2100, 10)

        # dropout layers
        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.1)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = F.relu(self.shrink1(x))
        x = F.relu(self.shrink2(x))
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = F.relu(self.conv4(x))
        # flatten image input
        x = x.view(-1, 14*9*50)
        # add dropout layer
        x = self.dropout1(x)
        # 1st hidden layer, relu activation
        x = F.relu(self.dense1(x))
        # dropout layer
        x = self.dropout2(x)
        # 2nd hidden --> output
        x = self.dense2(x)
        return x



In [ ]:
# create a complete CNN
cnet = cNet()
print(cnet)

# move tensors to GPU if CUDA is available
if use_cuda:
    cnet.cuda()

In [ ]:
use_cuda = torch.cuda.is_available()
if use_cuda:
    torch.cuda.ipc_collect()
    torch.cuda.empty_cache()
    torch.cuda.list_gpu_processes(), torch.cuda.memory_snapshot()

In [ ]:
#@title Skynet
out = output_volume((H,W),(5,5),2,2)
print("first in {} \t first out {}".format((H,W), out))
inn = out
out = output_volume(inn,(3,3),2,1)
print("second in {} \t second out {}".format(inn,out))
inn = out
out = output_volume(inn,(5,5),1,2)
print("third in {} \t third out {}".format(inn,out))
inn = (out[0]//2,out[1]//2)
out = output_volume(inn,(3,3),1,1)
print("fourth in {} \t fourth out {}".format(inn,out))
inn = (out[0]//2,out[1]//2)
out = output_volume(inn,(3,3),1,1)
print("fifth in {} \t fifth out {}".format(inn,out))
out0 = out[0]//2
out1 = out[1]//2
print("pool in {} \t pool out {}".format(out, (out0,out1) ))
print("flatout = {}".format(out0 * out1))
flatout = out0 * out1

class skyNet(nn.Module):
    ## TODO: choose an architecture, and complete the class
    def __init__(self):
        super(skyNet, self).__init__()
        ## Define layers of a CNN
        # number of nodes in each hidden layer

        # input layer ((H , W) -> con1)
        self.con1 = nn.Conv2d(3, 40, 5, stride=2, padding=2)
        self.con2 = nn.Conv2d(40, 20, 3, stride=2, padding=1)
        self.con3 = nn.Conv2d(20, 20, 5, stride=1, padding=2)
        self.con4 = nn.Conv2d(20, 30, 3, stride=1, padding=1)
        self.con5 = nn.Conv2d(30, 40, 3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2)
        self.flat = nn.Flatten()
        self.dense1 = nn.Linear(flatout*40, 5000)
        self.dense2 = nn.Linear(5000, 1000)
        self.out = nn.Linear(1000, 50)
        # dropout layers
        self.dropout2 = nn.Dropout(0.2)
        self.dropout3 = nn.Dropout(0.3)
        self.dropout4 = nn.Dropout(0.4)
        
    def forward(self, x):
        ## Define forward pass
        # convolutional layers
        x = F.relu(self.con1(x))
        x = F.relu(self.con2(x))
        x = self.pool(F.relu(self.con3(x)))
        x = self.pool(F.relu(self.con4(x)))
        x = self.pool(F.relu(self.con5(x)))
        # flatten input   
        #print (x.shape)
        x = self.flat(x)
        x = self.dropout4(x)
        # first hidden layer
        x = F.relu(self.dense1(x))
        x = self.dropout3(x)
        # second hidden layer
        x = F.relu(self.dense2(x))
        x = self.dropout2(x)
        # output layer (no activation)
        x = self.out(x)     
        return x

#-#-# Do NOT modify the code below this line. #-#-#





In [ ]:
skynet

In [ ]:
# instantiate the CNN
del skynet
skynet = skyNet()

# move tensors to GPU if CUDA is available
if use_cuda:
    skynet.cuda()
#skynet #(untrained!)

In [ ]:
locals()['skynet']

In [ ]:
torch.cuda.ipc_collect()
torch.cuda.empty_cache()
torch.cuda.list_gpu_processes()

In [ ]:
torch.cuda.reset_accumulated_memory_stats()

In [ ]:
vars()['output'].size()

In [ ]:
images, labels = test_data_iter.next()
skynet.eval()
output = skynet(images.cuda())
print("output\n",output)
criterion(output.cpu(), labels.cpu()).item()

In [ ]:
for p in skynet.parameters():
    n = p.size(0)
    torch.nn.init.normal_(p, mean=0.0, std=1/(n**0.5))

In [ ]:
#skynet.apply(custom_weight_init)
for p in skynet.parameters():
    print("{} {:.5f} {:0.5f}".format(p.size(0),p.mean().item(),p.std().item()))

In [ ]:
#skynet.apply(custom_weight_init)
skyopt = torch.optim.SGD(skynet.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

######### Train first!
skynet = train(3, loaders_scratch, skynet, skyopt, 
               criterion, use_cuda, 'skynet.pt')

In [ ]:
skynet = train(3, loaders_scratch, skynet, skyopt, 
               criterion, use_cuda, 'skynet.pt')

In [ ]:
# Sample guess

print (data[5].unsqueeze(0).shape, skynet(data[5].unsqueeze(0)).shape)
fig = plt.figure(figsize=(16,8),clear=True)
axes = fig.subplots(nrows=1, ncols=1, sharex=True, sharey=True)
im = data[5].unsqueeze(0)
ans = skynet(data[5].unsqueeze(0))

im=np.transpose(im.squeeze().detach().cpu().numpy(), (1,2,0))
axes.set_title(num2class[np.argmax(ans[0].detach().cpu().numpy())])
axes.set_xticks([])
axes.set_yticks([])
axes.imshow(im)

In [ ]:
[num2class[int(a)] for a in ans[:3]] #.sort() # = ans[0].detach().cpu().numpy()

__Question 2:__ Outline the steps you took to get to your final CNN architecture and your reasoning at each step.  

__Answer:__  

### * * * * *  **TRAIN** here!  * * * * *

(IMPLEMENTATION) Implement the Training Algorithm

Implement your training algorithm in the code cell below.  [Save the final model parameters](http://pytorch.org/docs/master/notes/serialization.html) at the filepath stored in the variable `save_path`.

In [ ]:
use_cuda = torch.cuda.is_available()
#torch.cuda.empty_cache()
torch.cuda.list_gpu_processes()
#torch.cuda.memory_snapshot()

# move tensors to GPU if CUDA is available
#if use_cuda:
#    skynet.cuda()
#skynet #(untrained!)

In [ ]:
#@ title train def
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    no_loss = 0

    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0

        ###################
        # train the model #
        ###################
        # set the module to training mode
        model.train()
        for i, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            ## TODO: find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - train_loss))

            # initialize optimizer variables to zero
            optimizer.zero_grad()
            # forward pass
            output = model(data)
            # calculate loss
            loss = criterion(output, target)
            print("raw loss: {:.6f} batch size: {} \tspin #{}  epoch {}".format(loss.item(), data.size(0), i, epoch))
            # backward pass
            loss.backward()
            # update parameters
            optimizer.step()
            # update training loss
            train_loss += loss.item()*data.size(0) 
            print("train loss: {:.6f}  batch loss: {:.6f} \tspin #{}  epoch {}".format(train_loss,loss.item()*data.size(0), i, epoch))        
        train_loss = train_loss/len(loaders['train'].dataset)

        ######################    
        # validate the model #
        ######################
        # set the model to evaluation mode
        model.eval()
        for data, target in loaders['valid']:
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            ## TODO: update average validation loss 
            # forward pass
            output = model(data)
            # compute batch loss
            loss = criterion(output, target)
            # update validation loss 
            valid_loss += loss.item()*data.size(0)     
        valid_loss = valid_loss/len(loaders['valid'].dataset)
        
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
                                                                    epoch, 
                                                                    train_loss,
                                                                    valid_loss
                                                                    ))

        ## TODO: if the validation loss has decreased, save the model 
        ##       at the filepath stored in save_path
        
        if valid_loss < valid_loss_min:
            print('>>> Loss decrease of {:.9f} <<<'.format(valid_loss_min - valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
            no_loss = 0
        else:
            no_loss += 1
        ## Early stopping. End training if loss no longer decreasing.
        if no_loss >= 3:
            print("No more loss. Terminating session.")
            break
        
    return model

### (IMPLEMENTATION) Experiment with the Weight Initialization

Use the code cell below to define a custom weight initialization, and then train with your weight initialization for a few epochs. Make sure that neither the training loss nor validation loss is `nan`.

Later on, you will be able to see how this compares to training with PyTorch's default weight initialization.

In [ ]:
# get weight mean 

def custom_weight_init(m):
    ## TODO: implement a weight initialization strategy
    for p in m.parameters():
        n = p.size(0)
        torch.nn.init.normal_(p, mean=0.5, std=1.0/(n**0.5))

In [ ]:
#-#-# Do NOT modify the code below this line. #-#-#
model_scratch.apply(custom_weight_init)
optimizer_scratch = get_optimizer_scratch(model_scratch)
model_scratch = train(10, loaders_scratch, model_scratch, optimizer_scratch,
                      criterion_scratch, use_cuda, 'ignore_scratch.pt')

### (IMPLEMENTATION) Train and Validate the Model

Run the next code cell to train your model.

###Scratch

In [ ]:
out = output_volume((H,W),(3,3),3,2)
print("first in {} \t first out {}".format((H,W), out))

print("\nflattened {}".format(out[0] * out[1]))
flatout = out[0] * out[1]

#@title define the mnNet architecture
class mnNet(nn.Module):
    ## TODO: choose an architecture, and complete the class
    def __init__(self):
        super(mnNet, self).__init__()
        ## Define layers of a CNN
        # number of nodes in each hidden layer

        # input layer ((H , W) -> nodes_1)
        self.con1 = nn.Conv2d(3, 3, 3, stride=3, padding=2)
        self.flat = nn.Flatten()
        self.den1 = nn.Linear(flatout*3, 1000)
        self.out = nn.Linear(1000, 50)
        # dropout layers
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.2)
        
    def forward(self, x):
        ## Define forward pass
        # convolutional layers
        x = F.relu(self.con1(x))
        # flatten input
                        #x = x.view(-1, 53868)
        x = self.flat(x)
        # first dropout layer
        x = self.dropout2(x)
        # first hidden layer
        x = F.relu(self.den1(x))
        # second dropout layer
        x = self.dropout1(x)
        # output layer
        x = F.relu(self.out(x))       
        return x

#-#-# Do NOT modify the code below this line. #-#-#

# instantiate the CNN
mn = mnNet()

# move tensors to GPU if CUDA is available
if use_cuda:
    mn.cuda()

#mn(data)

In [ ]:
#@title mn train
mn.apply(custom_weight_init)
mnopt= get_optimizer_scratch(mn)
epochs = 1 
optimizer = mnopt
criterion = nn.CrossEntropyLoss()

valid_loss_min = np.Inf 
    
train_loss = 0.0
valid_loss = 0.0

###################
# train the model #
###################
# set the module to training mode
mn.train()
for batch_idx, (data, target) in enumerate(trainloader):
    # move to GPU
    if use_cuda:
        data, target = data.cuda(), target.cuda()

    ## TODO: find the loss and update the model parameters accordingly
    ## record the average training loss, using something like
    ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - train_loss))

    # initialize optimizer variables to zero
    mnopt.zero_grad()
    # forward pass
    output = mn(data)
    #print("output shape",output.shape,"\ttarget shape",target.shape)
    # calculate loss
    loss = criterion(output, target)
    # backward pass
    loss.backward()
    # update parameters
    mnopt.step()
    # update training loss
    train_loss += loss.item()*data.size(0)

train_loss = train_loss/len(loaders_scratch['train'].dataset) 
print ("train_loss",train_loss)

In [ ]:
mn.apply(custom_weight_init)
mnopt= get_optimizer_scratch(mn)
epochs = 3 
optimizer = mnopt
criterion = nn.CrossEntropyLoss()
mn = train(3, loaders_scratch, mn.apply(custom_weight_init), 
           mnopt, criterion_scratch, use_cuda, 'mn.pt')

In [ ]:
#@title mnvalid
######################    
# validate the model #
######################
# set the model to evaluation mode
valid_loss = 0.0
mn.eval()
for batch_idx, (data, target) in enumerate(validloader):
    # move to GPU
    if use_cuda:
        data, target = data.cuda(), target.cuda()

    ## TODO: update average validation loss 

    # forward pass
    output = mn(data)
    # compute batch loss
    loss = criterion(output, target)
    # update validation loss 
    valid_loss += loss.item()*data.size(0)
    
# calculate average losses
valid_loss = valid_loss/len(loaders['valid'].dataset) 

# print training/validation statistics 
print('Epoch: {} \tValidation Loss: {:.6f}'.format(0, valid_loss))

### (IMPLEMENTATION) Test the Model

Run the code cell below to try out your model on the test dataset of landmark images. Run the code cell below to calculate and print the test loss and accuracy.  Ensure that your test accuracy is greater than 20%.

In [ ]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    # set the module to evaluation mode
    model.eval()

    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# load the model that got the best validation accuracy
#model_scratch.load_state_dict(torch.load('model_scratch.pt'))
#test(loaders_scratch, model_scratch, criterion_scratch, use_cuda)
test(loaders_scratch, mn, criterion_scratch, use_cuda)

---
<a id='step2'></a>
## Step 2: Create a CNN to Classify Landmarks (using Transfer Learning)

You will now use transfer learning to create a CNN that can identify landmarks from images.  Your CNN must attain at least 60% accuracy on the test set.

### (IMPLEMENTATION) Specify Data Loaders for the Landmark Dataset

Use the code cell below to create three separate [data loaders](http://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader): one for training data, one for validation data, and one for test data. Randomly split the images located at `landmark_images/train` to create the train and validation data loaders, and use the images located at `landmark_images/test` to create the test data loader.

All three of your data loaders should be accessible via a dictionary named `loaders_transfer`. Your train data loader should be at `loaders_transfer['train']`, your validation data loader should be at `loaders_transfer['valid']`, and your test data loader should be at `loaders_transfer['test']`.

If you like, **you are welcome to use the same data loaders from the previous step**, when you created a CNN from scratch.

In [ ]:
### TODO: Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes

loaders_transfer = loaders_scratch


### (IMPLEMENTATION) Specify Loss Function and Optimizer

Use the next code cell to specify a [loss function](http://pytorch.org/docs/stable/nn.html#loss-functions) and [optimizer](http://pytorch.org/docs/stable/optim.html).  Save the chosen loss function as `criterion_transfer`, and fill in the function `get_optimizer_transfer` below.

In [ ]:
## TODO: select loss function
criterion_transfer = nn.CrossEntropyLoss()

def get_optimizer_transfer(model):
    ## TODO: select and return optimizer
    return optim.SGD(vgg16.classifier.parameters(), lr=0.01)

### (IMPLEMENTATION) Model Architecture

Use transfer learning to create a CNN to classify images of landmarks.  Use the code cell below, and save your initialized model as the variable `model_transfer`.

In [ ]:
## TODO: Specify model architecture

model_transfer = None




#-#-# Do NOT modify the code below this line. #-#-#

if use_cuda:
    model_transfer = model_transfer.cuda()

__Question 3:__ Outline the steps you took to get to your final CNN architecture and your reasoning at each step.  Describe why you think the architecture is suitable for the current problem.

__Answer:__  

### (IMPLEMENTATION) Train and Validate the Model

Train and validate your model in the code cell below.  [Save the final model parameters](http://pytorch.org/docs/master/notes/serialization.html) at filepath `'model_transfer.pt'`.

In [ ]:
# TODO: train the model and save the best model parameters at filepath 'model_transfer.pt'



#-#-# Do NOT modify the code below this line. #-#-#

# load the model that got the best validation accuracy
model_transfer.load_state_dict(torch.load('model_transfer.pt'))

### (IMPLEMENTATION) Test the Model

Try out your model on the test dataset of landmark images. Use the code cell below to calculate and print the test loss and accuracy.  Ensure that your test accuracy is greater than 60%.

In [ ]:
test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)

---
<a id='step3'></a>
## Step 3: Write Your Landmark Prediction Algorithm

Great job creating your CNN models! Now that you have put in all the hard work of creating accurate classifiers, let's define some functions to make it easy for others to use your classifiers.

### (IMPLEMENTATION) Write Your Algorithm, Part 1

Implement the function `predict_landmarks`, which accepts a file path to an image and an integer k, and then predicts the **top k most likely landmarks**. You are **required** to use your transfer learned CNN from Step 2 to predict the landmarks.

An example of the expected behavior of `predict_landmarks`:
```
>>> predicted_landmarks = predict_landmarks('example_image.jpg', 3)
>>> print(predicted_landmarks)
['Golden Gate Bridge', 'Brooklyn Bridge', 'Sydney Harbour Bridge']
```

In [ ]:
import cv2
from PIL import Image

## the class names can be accessed at the `classes` attribute
## of your dataset object (e.g., `train_dataset.classes`)

def predict_landmarks(img_path, k):
    ## TODO: return the names of the top k landmarks predicted by the transfer learned CNN
    


# test on a sample image
predict_landmarks('images/test/09.Golden_Gate_Bridge/190f3bae17c32c37.jpg', 5)

### (IMPLEMENTATION) Write Your Algorithm, Part 2

In the code cell below, implement the function `suggest_locations`, which accepts a file path to an image as input, and then displays the image and the **top 3 most likely landmarks** as predicted by `predict_landmarks`.

Some sample output for `suggest_locations` is provided below, but feel free to design your own user experience!
![](images/sample_landmark_output.png)

In [ ]:
def suggest_locations(img_path):
    # get landmark predictions
    predicted_landmarks = predict_landmarks(img_path, 3)
    
    ## TODO: display image and display landmark predictions

    
    

# test on a sample image
suggest_locations('images/test/09.Golden_Gate_Bridge/190f3bae17c32c37.jpg')

### (IMPLEMENTATION) Test Your Algorithm

Test your algorithm by running the `suggest_locations` function on at least four images on your computer. Feel free to use any images you like.

__Question 4:__ Is the output better than you expected :) ?  Or worse :( ?  Provide at least three possible points of improvement for your algorithm.

__Answer:__ (Three possible points for improvement)

In [ ]:
## TODO: Execute the `suggest_locations` function on
## at least 4 images on your computer.
## Feel free to use as many code cells as needed.



---
## Load VGG16 as revgg

To define a model for training we'll follow these steps:
1. Load in a pre-trained VGG16 model
2. "Freeze" all the parameters, so the net acts as a fixed feature extractor 
3. Remove the last layer
4. Replace the last layer with a linear classifier of our own

**Freezing simply means that the parameters in the pre-trained model will *not* change during training.**

In [ ]:
# Load the pretrained models from pytorch
revgg = models.vgg16(pretrained=True)

# print out the model structure
print(revgg)

In [ ]:
# Freeze training for all "features" layers
for param in revgg.features.parameters():
    param.requires_grad = False
    print(param.size(), param.requires_grad) 
    
for clsfr in revgg.classifier:#[:6]:
    for prmtr in clsfr.parameters():
        prmtr.requires_grad = True
        print(prmtr.size(), prmtr.requires_grad)

for clsfr in revgg.classifier:
    for prmtr in clsfr.parameters():
        print(prmtr.size(), prmtr.requires_grad)  

---
### Final Classifier Layer

Once you have the pre-trained feature extractor, you just need to modify and/or add to the final, fully-connected classifier layers. In this case, we suggest that you repace the last layer in the vgg classifier group of layers. 
> This layer should see as input the number of features produced by the portion of the network that you are not changing, and produce an appropriate number of outputs for the flower classification task.

You can access any layer in a pretrained network by name and (sometimes) number, i.e. `vgg16.classifier[6]` is the sixth layer in a group of layers named "classifier".

In [ ]:
use_cuda = torch.cuda.is_available()

In [ ]:
n_inputs = revgg.classifier[6].in_features

# new layers automatically have requires_grad = True
fork_layer = nn.Linear(n_inputs, 50)
#last_layer = nn.Linear(50*len(classes), len(classes))

revgg.classifier[6] = fork_layer

# if GPU is available, move the model to GPU
if use_cuda:
    revgg.cuda()

# check to see that your last layer produces the expected number of outputs
print(revgg)

In [ ]:
# Check for all layers if trainable on training
for param in revgg.features.parameters():
    print(param.size(), param.requires_grad) 
print("\n")
for param in revgg.classifier.parameters():
    print(param.size(), param.requires_grad) 

In [ ]:
classier = revgg.classifier[6]
plist = [p for p in classier.parameters()]
plist[0].min(), plist[0].max(), plist[0].mean(), plist[0].std()

In [ ]:
#import torch.optim as optim

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.001
vgg_optimizer = optim.SGD(revgg.classifier.parameters(), lr=0.0005)

for p in revgg.classifier[6].parameters():
    #torch.nn.init.normal_(p, mean=0.667, std=n_inputs**-0.5)
    torch.nn.init.normal_(p, mean=0.5, std=0.125)

---
### Training



In [ ]:
#@title retrain def

def retrain(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns retrained pretrained-model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    no_loss = 0

    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0

        ###################
        # train the model #
        ###################
        # set the module to training mode
        model.train()
        for data, target in loaders['train']:
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()*data.size(0)       
        train_loss = train_loss/len(loaders['train'].dataset) 

        ######################    
        # validate the model #
        ######################
        # set the model to evaluation mode
        model.eval()
        for data, target in loaders['valid']:
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            output = model(data)
            loss = criterion(output, target)
            valid_loss += loss.item()*data.size(0)      
        valid_loss = valid_loss/len(loaders['valid'].dataset)

        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
                                                                                epoch, 
                                                                                train_loss,
                                                                                valid_loss))
        if valid_loss_min-valid_loss > 0.0005:
            print('>>> Loss decrease of {:.6f} <<<'.format(valid_loss_min - valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
            no_loss = 0
        else:
            no_loss += 1
        
        if no_loss >= 3:
            print("<<< No valid loss decrease. Terminating training. >>>")
            break

    return model

In [ ]:
# Turn off updates for first classifier layer parameters 
#for param in revgg.classifier[:5].parameters():
#    param.requires_grad = False
## turn up learning rate
vgg_optimizer = optim.SGD(revgg.classifier.parameters(), lr=0.001)
## re-retrain!
n_epochs = 5
revgg = retrain(n_epochs, loaders_scratch,
                revgg, vgg_optimizer, criterion, 
                use_cuda, 'revgg.pt')

---
### Testing

In [ ]:

# track test loss 
test_loss = 0.0
class_correct = list(0. for i in range(50))
class_total = list(0. for i in range(50))

revgg.eval() # eval mode

# iterate over test data
for data, target in testloader:
    if use_cuda:
        data, target = data.cuda(), target.cuda()
    output = revgg(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update  test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    pct, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy())\
              if not use_cuda\
              else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(data.size(0)):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1 

# calculate avg test loss
test_loss = test_loss/len(testloader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

## keep track
num2correct = dict()
#num2confidence = dict()

for i in range(50):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (num2class[i], 
                                                         100 * class_correct[i] / class_total[i],
                                                         class_correct[i], 
                                                         class_total[i]))                  
        #num2correct.update({i:class_correct[i]})
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (num2class[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (100. * np.sum(class_correct) / np.sum(class_total),
                                                      np.sum(class_correct), 
                                                      np.sum(class_total)))

In [ ]:
batch_guesses = np.array(output.tolist())
batch_guesses

In [ ]:
[np.max(gl) for gl in batch_guesses] == pct.tolist()

In [ ]:
[np.max(gl)/np.sum(gl) for gl in batch_guesses]

### Visualize Sample Test Results

In [ ]:
# obtain one batch of test images
try:
    images, labels = dataiter.next()
except NameError:
    dataiter = iter(testloader)
    images, labels = dataiter.next()
images.numpy()

# move model inputs to cuda, if GPU available
if use_cuda:
    images = images.cuda()

# get sample outputs
output = revgg(images)
# convert output probabilities to predicted class
pct, preds_tensor = torch.max(output, 1)
preds = np.squeeze(preds_tensor.numpy()) if not use_cuda else np.squeeze(preds_tensor.cpu().numpy())

# plot the images in the batch, along with predicted and true labels
fig = plt.figure(figsize=(18, 20))
for idx in np.arange(12):
    ax = fig.add_subplot(4, 12/4, idx+1, xticks=[], yticks=[])
    plt.imshow(np.transpose(images.cpu()[idx], (1, 2, 0)))
    ax.set_title("%5s (%2d%%)\n[%5s]" % (    num2class[preds[idx]],
                                             pct[i],
                                             num2class[labels[idx].item()]),
                                  color=("blue" if preds[idx]==labels[idx].item()\
                                                else "red"))

In [ ]:
num2class[labels[idx].item()]